In [1]:
from scipy.stats import beta
from scipy.optimize import minimize_scalar

In [9]:
def hdi(N,D,pd_estimate,α=0.05):

    a = D + 0.5
    b = N - D + 0.5

    # Anonymous function for a better readability
    pdf = lambda x: beta.pdf(x, a, b)
    cdf = lambda x: beta.cdf(x, a, b)
    inv_cdf = lambda x: beta.ppf(x, a, b)
    
    
    if a == b:
        lower_hdi = inv_cdf(α/2)
        upper_hdi = inv_cdf(1-α/2)
    
    elif a <= 1 and b > 1:
        lower_hdi = 0
        upper_hdi = inv_cdf(1-α)
    
    elif a > 1 and b <= 1:
        lower_hdi = inv_cdf(α)
        upper_hdi = 1
    
    else:
    
        def f(L):
            result = pdf(inv_cdf(1-α + cdf(L))) - pdf(L)
            return result**2
        
        res = minimize_scalar(
            f,
            bounds=(0,inv_cdf(α))
            )
        
        lower_hdi = res.x
        upper_hdi = inv_cdf(1-α + cdf(lower_hdi))
        #print(f(lower_hdi))

    print(f"LB HDI {(1-α)*100}% = {lower_hdi}, UB HDI {(1-α)*100}% = {upper_hdi}")

    
    ## Jeffrey Test
    flag1 = pd_estimate > lower_hdi

    # HDI conclusions assuming inputed α
    if flag1:
        print(f"HDI conclusions: a predicted probability of default lies above the lower bound of highest density interval. It indicates that we are not underestimating the real probability of deafult with a {(1-α)*100}% confidence. ")
    else:
        print(f"HDI conclusions: a predicted probability of default lies below the lower bound of highest density interval. It indicates that we are underestimating the real probability of deafult with a {(1-α)*100}% confidence. ")
    
    # P value conclusions
    p_value = cdf(pd_estimate)
    
    flag2 = p_value > 0.1
    flag3 = p_value <= 0.01 and p_value >= 0.05
    print("\nP-value conlusions:\nH0: PD > DR")
    
    if flag2:
        print("We are not rejecting null hypothesis. The predicted PD lies within a range of the Beta distribution. It indicates that we are not underestimating the real probability of default.")
    elif flag3:
        print("Additional testing recommended.")
    else:
        print("We are rejecting null hypothesis. The predicted PD does not lie within a range of the Beta distribution. It indicates that we are underestimating the real probability of default.")

    # Final decision on model
    print("\nDecision:")
    if flag1 and flag2:
        print("Model passed all tests.")
    elif flag1 or (flag2 or flag3):
        print("Additional testing recommended.")
    else:
        print("Model should be recalibrated.")
    

In [10]:
# Example of function call
hdi(86,5,0.0439)

LB HDI 95.0% = 0.01794796574275698, UB HDI 95.0% = 0.1146563539031649
HDI conclusions: a predicted probability of default lies above the lower bound of highest density interval. It indicates that we are not underestimating the real probability of deafult with a 95.0% confidence. 

P-value conlusions:
H0: PD > DR
We are not rejecting null hypothesis. The predicted PD lies within a range of the Beta distribution. It indicates that we are not underestimating the real probability of default.

Decision:
Model passed all tests.
